In [1]:
import pandas as pd
import os
import datetime
import json
from functools import reduce

In [2]:
ROOT_DIR = r"C:\dev\backend\tasks\samples\cmots"
asof_date = datetime.date(2023, 3, 2)
DIR_PATH = os.path.join(ROOT_DIR, asof_date.strftime("%d%m%Y") + '_ulip_master')

# Files needed to be refreshed from CMOTS ftp : inscomp.csv, insmplan.csv, insfundm.csv, FundCode_SFIN_Mapping_15Mar2023.xlsx

DIR_PATH

'C:\\dev\\backend\\tasks\\samples\\cmots\\02032023_ulip_master'

In [3]:
def get_df(dir_path, file_name, asof_date, lst_col, lst_drop_col):
    file_name = os.path.join(dir_path, file_name)
    print(file_name)
    if os.path.isfile(file_name):
        print('Reading the file into df...')
        df = pd.read_csv(file_name, sep ='|', names=lst_col, encoding= 'unicode_escape' )
        df.iloc[:, -1] = df.iloc[:, -1].str.split('<</EOR>>', expand=True)
        # print(df.head())
        df = df.drop(lst_drop_col, axis=1)
    else:
        df = pd.DataFrame()
    return df

In [4]:
# 1] inscomp.csv
lst_col = ["INSCODE","INSURER","INSNAME","REGCODE",
           "REGDATE","CLASS","FUNDWISE","FLAG"]
lst_used = ["INSCODE", "INSURER", "INSNAME", "REGCODE", "REGDATE"]
lst_drop = [item for item in lst_col if item not in lst_used]

df_inscomp = get_df(DIR_PATH, "inscomp.csv", asof_date, lst_col, lst_drop)
df_inscomp.sort_values("INSCODE", inplace=True)
df_inscomp.head()

C:\dev\backend\tasks\samples\cmots\02032023_ulip_master\inscomp.csv
Reading the file into df...


,INSCODE,INSURER,INSNAME,REGCODE,REGDATE
0,12024,Life Insurance Corporation of India,LIC,512,01/09/1956
1,20657,Birla Sun Life Insurance Company Ltd,Birla Sun Life Insurance,109,31/01/2001
2,20878,Max Life Insurance Company Ltd,Max Life Insurance,104,15/11/2000
3,21997,HDFC Standard Life Insurance Company Ltd,HDFC Life Insurance,101,23/10/2000
4,22005,ICICI Prudential Life Insurance Company Ltd,ICICI Pru Life Insurance,105,24/11/2000


In [5]:
# 2] insmplan.csv
lst_col = ["INSCODE","MPLANCODE","MPLANNAME","CATEGORY","AVAIL","RELATE","STATUS","FLAG"]
lst_used = ["MPLANCODE","MPLANNAME","CATEGORY","AVAIL","RELATE","STATUS"]
lst_drop = [item for item in lst_col if item not in lst_used]

df_insmplan = get_df(DIR_PATH, "insmplan.csv", asof_date, lst_col, lst_drop)
df_insmplan.sort_values("MPLANCODE", inplace=True)
df_insmplan.head()

C:\dev\backend\tasks\samples\cmots\02032023_ulip_master\insmplan.csv
Reading the file into df...


,MPLANCODE,MPLANNAME,CATEGORY,AVAIL,RELATE,STATUS
1108,5001,Aviva Life Long - With Profits,Protection Plan,INDIVIDUAL,ULIP,C
1109,5002,Aviva LifeShield,Protection Plan,INDIVIDUAL,TRADITIONAL,C
1110,5003,Aviva Easy Life Plus,Protection Plan,INDIVIDUAL,ULIP,C
1111,5004,Aviva Life Bond 5,Protection Plan,INDIVIDUAL,ULIP,C
1112,5005,Aviva LifeSaver - With Profits,Protection Plan,INDIVIDUAL,ULIP,C


In [6]:
# 3] insfundm.csv
lst_col = ["INSCODE","FUNDCODE","FUNDLNAME","FUNDSNAME","COMMONCODE","AVAIL","CREATION","PENSION","PORTCODE","FLAG"]
lst_used = ["FUNDCODE","FUNDLNAME","FUNDSNAME","PORTCODE"]
lst_drop = [item for item in lst_col if item not in lst_used]

df_insfundm = get_df(DIR_PATH, "insfundm.csv", asof_date, lst_col, lst_drop)
df_insfundm.sort_values("FUNDCODE", inplace=True)
df_insfundm.head()

C:\dev\backend\tasks\samples\cmots\02032023_ulip_master\insfundm.csv
Reading the file into df...


,FUNDCODE,FUNDLNAME,FUNDSNAME,PORTCODE
73,2038,Birla Sun Life Assure Fund,Assure Fund,2038
74,2039,Birla Sun Life Protector Fund,Protector Fund,2039
75,2040,Birla Sun Life Builder Fund,Builder Fund,2040
76,2041,Birla Sun Life Balancer Fund,Balancer Fund,2041
77,2042,Birla Sun Life Enhancer Fund,Enhancer Fund,2042


In [7]:
# 4] iulipfundop.csv
lst_col = ["INSCODE","MPLANCODE","FUNDCODE","TYPE","FLAG"]
lst_used = ["MPLANCODE","FUNDCODE","FLAG"]
lst_drop = [item for item in lst_col if item not in lst_used]

df_iulipfundop = get_df(DIR_PATH, "iulipfundop.csv", asof_date, lst_col, lst_drop)
df_iulipfundop.sort_values("FUNDCODE", inplace=True)
df_iulipfundop.head()

C:\dev\backend\tasks\samples\cmots\02032023_ulip_master\iulipfundop.csv
Reading the file into df...


,MPLANCODE,FUNDCODE,FLAG
1693,6505,2038,A
1536,6397,2038,A
160,5079,2038,A
167,5078,2038,A
1547,6398,2038,A


In [8]:
# 5] FundCode_SFIN_Mapping_15Mar2023.xlsx
file_name = os.path.join(DIR_PATH, "FundCode_SFIN_Mapping_15Mar2023.xlsx")
df_sfin = pd.read_excel(file_name)
df_sfin.head()

,INSCODE,FUNDCODE,SFIN
0,12024,2084,ULIF 001 02/02/01 LICBMA+SEC 512
1,12024,2085,ULIF 002 02/02/01 LICBMA+BAL 512
2,12024,2086,ULIF 003 02/02/01 LICBMA+RSK 512
3,12024,2087,ULIF 001 04/03/05 LICFUT+BND 512
4,12024,2088,ULIF 002 04/03/05 LICFUT+INC 512


In [10]:
# 6] iplanclass.csv
lst_col = ["INSCODE","MPLANCODE","RELATE","UIN","CATCODE","CATNAME",
           "SUBCATCODE","SUBCATNAME","SUBOCATCODE","SUBOCATNAME",
           "OPTION1","OPTION2","OPENDATE","CLOSEDATE","STATUS",
           "OPENDATE1","CLOSEDATE1","OPENDATE2","CLOSEDATE2","ROLLOVER","REMARKS","FLAG"]
lst_used = ["MPLANCODE","UIN"]
lst_drop = [item for item in lst_col if item not in lst_used]

df_iplanclass = get_df(DIR_PATH, "iplanclass.csv", asof_date, lst_col, lst_drop)
df_iplanclass.head()

C:\dev\backend\tasks\samples\cmots\02032023_ulip_master\iplanclass.csv
Reading the file into df...


,MPLANCODE,UIN
0,5001,122L003V02
1,5002,122N019V01
2,5003,122L044V02
3,5004,122L025V02
4,5005,122L005V02


In [11]:
df_final = pd.merge(df_sfin, df_iulipfundop, on="FUNDCODE")
df_final = pd.merge(df_final, df_insfundm, on="FUNDCODE")
df_final = pd.merge(df_final, df_insmplan, on="MPLANCODE")
df_final = pd.merge(df_final, df_iplanclass, on="MPLANCODE")
df_final[["INSCODE","MPLANCODE","MPLANNAME","FUNDCODE","FUNDLNAME","FUNDSNAME","SFIN","UIN","PORTCODE","CATEGORY","AVAIL","RELATE","STATUS","FLAG"]].head()
df_final["MPLANNAME"] = df_final["MPLANNAME"].str.replace("'","''")
df_final["FUNDLNAME"] = df_final["FUNDLNAME"].str.replace("'","''")
df_final["FUNDSNAME"] = df_final["FUNDSNAME"].str.replace("'","''")
df_final["CATEGORY"] = df_final["CATEGORY"].str.replace("'","''")
#df_final[""] = df_final[""].replace("'","''")

df_final.to_excel(r'C:\dev\cmots\sql\01_CMOTS_ULIP_Data_Insert_Script.xlsx')

In [41]:
# 5] inspf.csv
lst_col = ["INSCODE","PORTCODE","FUNDCODE","INVDATE","TYPECODE","STYPECODE",
           "DEBTYPCODE","RATINGCODE","RATING","PERIOD","PEROFCODE","CORPUS",
           "MKTVALUE","CMSECTOR","REPSECTOR","ASECT_CODE","AMFISECTOR",
           "INV_NO","CO_CODE","CO_NAME","NO_SHARES","PERC_HOLD","INTPER","EDATE",
           "GROUP","COMCODE","COMNAME","TCODE","TNAME","TOPPF","LNAME","REPNAME",
           "DERIV_MONTH","CTRY_CODE_N","CTRY_CODE_A","CTRY_NAME","FLAG"]
lst_used = ["INSCODE","PORTCODE","FUNDCODE","INVDATE","RATING","PERIOD","CORPUS",
            "MKTVALUE","CO_CODE","CO_NAME","NO_SHARES","PERC_HOLD","INTPER",
            "COMCODE","COMNAME","LNAME","REPNAME","FLAG"]
lst_drop = [item for item in lst_col if item not in lst_used]

df_inspf = get_df(DIR_PATH, "inspf.csv", asof_date, lst_col, lst_drop)
df_inspf.sort_values("FUNDCODE", inplace=True)
df_inspf["CO_NAME"] = df_inspf["CO_NAME"].str.replace("'", "''")
df_inspf["COMNAME"] = df_inspf["COMNAME"].str.replace("'", "''")
df_inspf["LNAME"] = df_inspf["LNAME"].str.replace("'", "''")
df_inspf["REPNAME"] = df_inspf["REPNAME"].str.replace("'", "''")
#df_inspf[""] = df_inspf[""].str.replace("'", "''")
#df_inspf.head()

df_inspf.to_excel(r'C:\Users\Kamle\Desktop\Others\01_CMOTS_ULIP_Portfolio_Data_Insert_Script.xlsx')

C:\dev\backend\tasks\samples\cmots\02032023_ulip_master\inspf.csv
Reading the file into df...


C:\Users\Kamle\AppData\Local\Temp\ipykernel_64948\731001459.py:6: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name, sep ='|', names=lst_col, encoding= 'unicode_escape' )
